# YOLO: You Only Look Once

## Introduction

Titles of the articles are You only look once: Unified Real-Time Object detection whose authors are Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi. The main purpose of this article is to present a new object detection algorithm (YOLO) with a different approach and other two are YOLO9000: Better, Faster and stronger and YOLOv3: An Incremental Improvement whose authors are Joseph Redmon and Ali Farhadi. The purpose of these two articles is just the improvement which covers the loopholes of the presented algorithm.

## Content

### Overview

YOLO has a simple and straightforward process in detecting the object as mentioned in the paper as follows
*	Re-size the image: Dividing the image into S*S Grids.
*	Runs a single Convolutional neural network: The bounding boxes B are predicted at different locations in the image and predict the class of the object.
*	Performs Non-Max suppression: It is a process where most of the bounding boxes are supressed which has not-maximum Intersection over Union.
* **Intersection over union**: This convey how much of the ground truth has common with the predicted bounding box. this has a maximum limit of 1.
                Intersection of union = Area of overlap/area of union
The above-mentioned steps are to be considered as a broader and high-level view of YOLO algorithm while we have more depth to be explored which leads to


### UNDERSTANDING YOLO

In order to understand YOLO algorithm there are some concepts to be known like Object localisation, Object classification, Bounding box specification, Non-Max suppression, Loss functions.

* **object Classification**: It is a pre-trained method where a particular object is classified by passing them to CNN’s with appropriate labels.
The pictures (Training set) are fed into the convnets and output is given by SoftMax function where it gives the output in form of a matrix.

    * **Example** :  if a self-driving car is being built it has to detect various object on the road like 
     1. Pedestrians 
     2. Car 
     3. Motorcycle 
     4. Background
     


       If the object is car then the result matrix will be
       
 \begin{equation} (0,1,0,0)  =   (c1,c2,c3,c4 )  \end{equation}
 

* **Object Localization**: The term object localization refers to figuring out where in the picture the object present. These are predicted by sending the images to convnets with already pre-defined ground truth bounding box done manually. bounding box have 4 components $ b_{x},b_{y},b_{w},b_{h} $ where $ b_{x} $  and $b_{y} $ are the midpoint of the bounding box, $ b_{w} $ and $ b_{h} $ are the width and height of bounding box. 
    
     ![image.png](Car.png)
                 Fig-2:Bounding box ground truth of he car
      
    So, input will be in the form of matrix which has 4 components namely  \begin{bmatrix}b_{x}\\b_{y}\\b_{w}\\b_{h} \end{bmatrix}
    

* **Object Classification with localization**: In this process both classification and localization happens simultaneously which results in the following matrix including 4 classes (pedestrians, Car, Motorcycle, background) and $ b_{x}, b_{y}, b_{h} $ and $ b_{w} $ there will be an extra component with these called $ p_{c} $ which conveys the probability of the object present in the picture
    The final output (target label y) will be
   
  \begin{bmatrix} p_{c}\\b_{x}\\b_{y}\\b_{h}\\b_{w}\\c_{1}\\c_{2}\\c_{3}  \end{bmatrix}

    If $ p_{c} = 0 $ then other numbers in the output will not be cared as object is not present in the picture.
    We don’t have $ c_{4} $ because if the remaining three classes are 0 then indirectly classified as fourth class (Background).
    Example: as per the given example above in fig 2 the input must be following 

\begin{bmatrix}1\\b_x\\b_y\\b_h\\b_w\\0\\1\\0\end{bmatrix}

* **Loss Functions**:  These can be defined as the function which calculates the loss (difference) between the ground truth and predicted bounding box.
   
   **Sum-Squared Error**: The multipart function used in yolo stated as equation 3
    ![LossFunction](LossFunction1.png)
    S = number of grids, B = Bounding boxes
    
    **Classification loss**:
    ![image.png](LossFunction2.png)
    $ 1_{i}^{obj} $= 1 if object appears in cell I, otherwise 0.
    
    $\hat{p_{i}}(c) $= the conditional class probability of class c in cell i.
    
    **Localization loss**: only consider the boxes containing object 
    ![image.png](LossFunction3.png)
    $ 1_{i}^{obj} $ = 1 if $ j^{th} $ boundary box in cell I is responsible for predicting the boundary box, otherwise 0
   
   $ λ_{coord} $ increase the weight for the loss in boundary box co-ordinates

    $(x_{i}, y_{i},w_{i},h_{i})$ = ground truth co-ordinated of bounding box
    
    $(\hat{x}_{i}, \hat{y}_{i},\hat{w}_{i},\hat{h}_{i})$ = predicted bounding box co-ordinates.
    
    Yolo uses square root for width and heigh instead of actual width and height and $ λ_{coord} $ because it doesnt want to calculte equal errors for small and large boxes where a small error in large box doesnt make difference but it makes a huge difference for small box
    
    **Confidence Loss**: if the object is in the box then
     ![image.png](LossFunction4.png)
      
     $ C_{i} $ = confidence score of box j in cell i

     $ 1_{i}^{obj} $ = 1 if $ j^{th} $ boundary box in cell I is responsible for predicting the boundary box, otherwise 0
            When the object is not present
    ![image.png](LossFunction5.png)
    $ λ_{noobj} $= wheighs down the loss when detcting background
    
    The reason for using sum squared error is it weighs the localization error eqaully with classification error and in many grid cells do not contain object which pushes the confidnece score to 0 which can lead to model instability, to remedy this problem increased loss of bounding box prediction and decresed loss of conifdence prediction for boxes do not contain objects with help of two parametses λ_noobj.[4]
    
    Mainly this loss function is used to 

*    **Bounding Box specification and prediction**: All the bounding boxes are predicted by performing object classification and localization on each cell, which results in total S$*$S$*$(B$*$5+C) tensor
    
    Example: a Picture which has been divided into 7 grids and has 2 bounding boxes for each grid with 10 classes the output predictions will be  7$*$7$*$20 tensor.
    all the co-ordinates of the bounding box are normalised so that they lie between 0 and 1. 
    
       **Non_Maxima Supression**: This is a process of Supressing the prediction which has minimun confidnece scores.

    **Working**:
      Each bounding box has following 
      \begin{bmatrix}p_c\\b_x\\b_y\\b_h\\b_w\end{bmatrix}
      * Discard all the boxes which p_c is less than 0.6
      * Pick the ones which have highest p_c and output as prediction
      * Discard any other with IOU is less than 0.5
   
   ![image.png](Architecture.png)
   
        
        The Architecture of yolo which has 24 conv layers for which first 20 layers are pretrained with ImageNet dataset followed by Average-pooling and Fully connected layer and then converted the model for detection by adding 4 conv layers and 2 FC layers with randomly initialised weights with also increased resolution to 448*448 for detection.

## Innovation

YOLO was developed in 2015 since then innovation happened till recently with 2 more Iteration to improve the algorithm and cover the loopholes and errors that YOLO had at that time

Innovation happened by introducing 

YOLO9000 and YOLOv3
    
* **YOLO9000**
        Improvemnets made for this version as follows:
        * Batch Normalization
        * High Resolution Classifier
        * Convolutions with Anchor Box
        * Dimesion Clusters
        * Direct Location Prediction
        * Fine Grained Features
        * Multi-Scaling Training
    **Batch Normalisation**:Batch normalization happens after convolutional layer before activation function. The main purpose to use this to keep the data clean every layer so that data stop detriorating each layer. This ideally puts the input layer between 0 and 1(Not too much variance) which has improved mean average precision by 2%. it also helps regularize model.[6]

    **High-Resolution Classifier and fine grained feature**: YOLOv2 first fine tune classification network with 448*448 for 10 epochs on imagenet which gives network time to adjust its filters to work better on higher resolution we then fine tune the resulting network on detction.This helps to increase mean average precsion by 4%.
	Idea of fine grained feature is to reduce the spatial dimensions and increase the depth of the image which gives access to wide range of features to capture details without increasing the spatial size.

    **Convolutional with Anchor boxes**: The advantage of predicting the bounding boxes using anchor boxes is instead of predicting the co-ordinates they can predict the offsets with respect to anchor boxes with which diversity among predictions can be maintained by focussing on each shape. Fully connected layers are removes and anchor boxes are used and pooling layer is elimintaed to have high resolution ouptut. 
    
    Operation on 416*416 happens because of the odd number of saptial dimension in feature map so that there would be a single cell in the middle which has high chances for the presnece of large objects in middle of the image.
    
    Now class prediciton and abject detection moved form grid level to anchor box level where measuring IOU with the ground truth and proposed bounding box still remains.

     **Dimesion Cluster**:Dimension clusters solves first issue using anchor 	boxes which is picking anchor boxes randomly by hand based on the shapes of different kind of objects, but instead of 	choosing by hand running k-means clustering on training set bounding boxes to automatically find good anchor boxes.Using standard k-means with euclidean distance generates greater errors for larger boxes than smaller ones which can be solved by IOU relative to each other. K=5 is considered as a good tradeoff between model complexity and high recall. Ideally decided 5 bounding boxes for eahc cell would be good decision for better 	results.

    **Direct Location Prediction and multi scale training**:Direct location prediction solves second issue using anchor boxes with YOLO is model instability. The network predicts 5 bounding boxes per each grid cell which has 5 co-ordinates $ t_{x}, t_{y},t_{w},t_{h} $  and $ t_{0} $ and if the cell is offset from top-left corner by $ (c_{x}, c_{y}) $ and anchor box has width and height of $ p_{w} $ and $ p_{h} $ then bounding box prediction will be
![image.png](DirectLocation.png)

    The co-ordinates of the bounding box will be the sum of offset range of the grid cell and the co-ordinate applied with sigoid function which coresponds to [0,1].

    The width and height of the bounding box will be the product of height and width of anchor box to the corresponding bounding box to the exponents.

![image.png](DirectLocation2.png)
    
    Instead of having fixed input image size Yolov9000 runs in different images sizesand chooses random image for every few batches and downsamples it by factors of 32 which forces network to learn across different dimensions. And works as data augmnetation.
    Yolo9000 can predict 9000 classes because of hierachial classification where yolo combines all the labels of different datasets to form WorkdTree.

* **YOLOv3**
    
    Improvements made to the previous versions affects the following
    
    **Class Prediction**: Till now the class predictions meant to be mutually exclusive where the maximum probaility of a certain class will be 1 with softmax function. But in yolov3 replaces softmax with independent logistic classifiers to claculate the likeliness of the imput belongs to specific class.instead of using mean squared error for classification loss binary cros entropy is used for each label which also reduces computation.
    Binary Cross entropy:
    ![image.png](BinaryCE.png)
    
    Where y is the label and p(y) is the predicted probability of a specific class.
   
   If y=1 it adds the log(p(y)) to the loss, conversely it adds (1-p(y)) if y=0.[5]

    **Bounding box prediction**:this version of yolo predicts each bounding objectness score using logistic regression, the value will be 1 if this bounding boxprior overlaps with ground truth by any other prior, if the prior overlaps a bit with the ground truth surpassing some treshold then it will be ignored.if no bounding box priors assigned to ground truth then it incure no loss or class prediction.
    
    **Prediction across sales using Feature pyramid Networks**: 
           * This version of yolo makes predictions at 3 different scales In the last feature map and goes back 2 layers back and upsamples it by 2 retrieves the feature map and merge it with upsampled feature map using element-wise addition and apply conv nets to make second set prediction
           
           * It repeates the above step until the result feature map has high-level semantic information. Yolov3 applies k-means cluster ot determine and pre-selects the cluster.

    **Darkenet-53 replaces darknet-19 for the sake of speed delivering same accuracy, this pseed is achieved because darknet-52 has less billion floating point operations which save stime and computation and composed of 3*3 and 1*1 filter decreases size but increases depth of the image.**

## Technical quality

These papers are embeded with a high level of technicality. although these articles have around 5-7 pages the rectification of technical errors and improvements conveys the attention to technical aspects of the algorithm. The positive side of these papers is the comparision of this algorithm with different level state of the art algorithms with different datasets and its own previous versions showing the technical imporvement in terms of speed,accuracy,computation and complexity. 
Statistics mentioned in the papers show us show us that the by each iteration how technical quality improved with time.
On the other side Yolo: An Incremental improvement paper stated the drwbacks on its algorithm where it struggles to detect medium or larger objects for which reasons or explanations are not disclosed, whereas in yolo:you look only once has stated its drawbacks and gave reasons of occurance.

If the reasons for the yolov3 drawbacks would have been given then there would be a huge scope of research by different kinds of users which can lead to improved technicality.
drawbacks of YOLO were considered for the seoncd and third versions where the drawbacks have become the working technical aspects until YOLOv3 but in this iterating process the strong technical aspects of YOLOv1 have become drawbacks of YOLOv3.

Another Topic worth discussion is the introduction to anchor boxes, as mentioned above in innovation using anchor boxes in YOLO creates two issues which are model instability and picking priors by hand which have been dealt reasonably in YOLO9000 resulting a bit of complication, after later versions as mentioned above paragraph we found out YOLOv3 have succesfully overcome its drawbacks but created a new ones which used to be perfectly functional which brigs a question "whether using Anchor boxes is a right choice or not?"


## Application and X-factor

The combined application and technique of this topic is to get the input and interpret it and produce th eoutput in the form of matrix which can be later plotted as co-ordinates. However this algorithm is used to detect the objects in the image and classify them which has been done in different and more efficent ways in those three papers.
The main drawbacks after first version have been caught that the algorithm cannot detect small objects and struggles to generalise the objects in new or unusual aspect ratios and training of loss function which approximates detection performance leads greater effect on IOU.
However in the later version use of anchor boxes,changes in the architecture and many other chnages have covered the errors and improved the accuracy and speed of the algortihm which led to increase in efficiency of the algortihm.
The third iteration of the algorithm has an X-Factor of having a bit bigger network but have managed to achieve good accuracy and speed acompared to other two versions and yolov3 cannot detect large or medium size object which is quite opposite to the first version of YOLO, we can actually consider that yolov3 is a actual incremental imrovement that doesn’t much have any drwabacks or considerable errors.


# Presentation

The three papers had a decent structure and reasonably good explanation of three versions of YOLO. The structre includes abstract, introduction and explanation of the algorithm followed by limitations and coclusion whereas the later two had abstract, introduction and the imporvements aspects, comparisions and conclusions. All the comparisions of accuracy with other algorithms had basis and proofs.
However the YOLO:you only look once article is bit hard to imagine and understand as it consist with new approach to object detection and other two articles are mere improvements of the previous ones with more performance statistics. Some pre-requisites for this articles can be some understanding about object detction,localization and classification.Very high-level of technical terminology used in these three articles which can be hard and ambiguous  to understand some of the core concepts of this algorithm.


## References

[1]	Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi, ’You Only Look Once: Unified, Real-Time Object Detection’, 2016 <https://arxiv.org/pdf/1506.02640.pdf>

[2]	Joseph Redmon, Ali Farhadi, ’ YOLO9000: Better, Faster, Stronger’ 2016 <https://arxiv.org/pdf/1612.08242.pdf>

[3]	Joseph Redmon, Ali Farhadi, ‘YOLOv3: An Incremental Improvement’ 2018 <https://arxiv.org/pdf/1804.02767.pdf>

[4]	Jonathan Hui,’ Real-time Object Detection with YOLO, YOLOv2 and now YOLOv3 , 2018         <’https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088>

[5]	Daniel Godoy, ‘Understanding binary cross-entropy / log loss: a visual explanation’, 2018    <https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a>

[6]	machineThink, ‘Real-time object detection with YOLO’,2017
<https://machinethink.net/blog/object-detection-with-yolo/>


In [ ]:
Link to Github: